<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
# Introduction to Databases: Relational, Query and Web Application


## Overview

- We will build the core elements of four applications to understand the practical applications of DBs
    1. Interactive, transactional web application.
    2. Data analysis, analytics and decision support.
    3. Machine learning.
    4. (Social) graph analysis.
    

- All domains are very complex and complete applications are large and require mastering many concepts other than databases. So, the apps will be simplistic and what we do will focus on the database usage.


- Lecture 3 covers the following topics:
    1. Structure of an interactive, web application.
    1. Introduction to best practices for writing code that interacts with databases.
    1. Comparison of implementing the data layer using [comma separated value](https://en.wikipedia.org/wiki/Comma-separated_values) files and a relational database.
    1. Introduction to relational theory.
    1. Introduction to data-modeling.
    1. Introduction to Structured Query Language (SQL) as a realization and extension of relational theory.
        1.Finalize definition of homework/project 1.

## Our Application $-$ Reminder and Preview

### Application Overview and User Experience

#### Application Functionality

- We have data on Major League Baseball teams and players.


- We want to allow users to:
    - Find, query and display information.
    - Update information.


- Our design methodology is "Data Out."
    - We start with the data and datamodel.
    - Based on our understanding of the application objectives, how do we need to structure the data and what operations do we need to support?


#### User Experience

- In large projects, there are several teams.
    - The data layer teams is usually different from UI team.
    - I want to give you a preview of what the UI/UX might look like.
<br><br>

| <img src="../images/L4_search_page.jpeg"> |
| :---: |
| __Example User Interface__ |


- The UI task is to design and implement a baseball dashboard with three UI controls: 
    - Select the resource(s) to query or update (Players, Batting, Appearances)
    - Find an instance of the resource if you know the identifying fields.
    - Create an _ad hoc_ query to locate resource instances based on columns and values. Choose which columns you want in responses.
    
- Second phase will add support for create, update and delete.
    

- From [Techopedia](https://www.techopedia.com/definition/30581/ad-hoc-query-sql-programming): "As the word 'ad hoc' suggests, this type of query is designed for a 'particular purpose,' which is in contrast to a predefined query ... An ad hoc query does not reside in the system for a long time and is created dynamically on demand by the user."
    - You can write a program or function for a predefined query, e.g. ```findByLastName(ln).``` The set of functions you write implements the predefined queries.
    - _Ad hoc_ allows the user to enter arbitrary queries you did no anticipate.
    
    
- But again, we will start with the data layer.

#### Application Tiers/Layers

| <img src="../images/webapp.jpeg"> |
| :---: |
| __Application Structure__ |

##### UI Layer

- There are design patterns or best practices for designing user interfaces:
    - [Model-View-Controller](https://en.wikipedia.org/wiki/Model%E2%80%93view%E2%80%93controller)
    - [Model-View-Viewmodel](https://en.wikipedia.org/wiki/Model%E2%80%93view%E2%80%93viewmodel)
    

- The _Model_ is the data and operations on the data, and is independent of the UI specifics.


- The _View_ is the visual representation, display and interaction between the user and the application.


- The _Controller_ or _Viewmodel_ is the code that binds the View to the Model.

##### Business/Application Layer

"Application Tier: Also called the middle tier, logic tier, business logic or logic tier, this tier is pulled from the presentation tier. It controls application functionality by performing detailed processing." (https://www.techopedia.com/definition/24649/three-tier-architecture).


- The definition is a little vague. Consider the business logic of a transfer between two accounts. The application can process a transfer if:
    - Both accounts exist.
    - The transfer amount is ```> 0``` and less than the bank's transfer limit.
    - The remaining balance in the source account is
        - ```>= (0 - over_draft_limit)``` if the source is a checking account.
        - ```>= minimum_balance``` if the source is a savings account.
    - ```over_draft_limit``` and ```minimum_balance``` may depend on information about the customer, country, etc.
    
    
- This application logic applies independently of
    - The UI (web app, mobile device, ATM, ...)
    - The data storage and data format.
    
    
##### Data Tier/Layer

- The data tier is:
    - The database systems the manage the data.
    - A programming abstraction/API that the business logic uses to access the data.
    
    
- The data tier isolates the application logic from the details in and changes of the underlying data tier.


##### Separation of Concerns

"In computer science, separation of concerns (SoC) is a design principle for separating a computer program into distinct sections, such that each section addresses a separate concern. ...

Layered designs in information systems are another embodiment of separation of concerns (e.g., presentation layer, business logic layer, data access layer, persistence layer). 

The value of separation of concerns is simplifying development and maintenance of computer programs. When concerns are well-separated, individual sections can be reused, as well as developed and updated independently. Of special value is the ability to later improve or modify one section of code without having to know the details of other sections, and without having to make corresponding changes to those sections." (https://en.wikipedia.org/wiki/Separation_of_concerns)


## Data Layer

### Physical Data Model

- We will write a data access layer to CREATE-RETRIEVE-UPDATE-DELETE data from CSV files.


| <img src="../images/lahman2017-1.jpeg"> |
| :---: |
| __Reminder: Data Model__ |



- The first step is a Data Access Object \(DAO\).
    - We will do a general class that is configurable.
    - The configuration data is metadata defining the CSV file and collection.
    

- The basic operations will be:
    - ```findByPrimaryKey```
    - ```findByTemplate```
    - ```update```
    - ```create```
    - ```delete```

<br>
    
    
| <img src="../images/dao.png"> |
| :---: |
| __Reminder: Data Model__ |
    

### What happened?

#### Database Connection

"In computer science, a database connection is the means by which a database server and its client software communicate with each other. The term is used whether or not the client and the server are on different machines.

The client uses a database connection to send commands to and receive replies from the server." (https://en.wikipedia.org/wiki/Database_connection)

| <img src="../images/db-connection.jpg"> |
| :---: |
| __Database Connection__ |

1. Created a database connection using a client helper library.
<br><br>
1. Sent a command in a _Data Manipulation Language_ to the database over the connection as a string.
```
SELECT 
        Batting.playerID, 
        (SELECT People.nameFirst FROM People WHERE People.playerID=Batting.playerID) as first_name, 
        (SELECT People.nameLast FROM People WHERE People.playerID=Batting.playerID) as last_name, 
        sum(Batting.h)/sum(batting.ab) as career_average, 
        sum(Batting.h) as career_hits, 
        sum(Batting.ab) as career_at_bats,
        min(Batting.yearID) as first_year, 
        max(Batting.yearID) as last_year 
        FROM 
        Batting 
        GROUP BY 
        playerId 
        HAVING 
        career_at_bats >=200 AND last_year >= 1960 
        ORDER BY 
        career_average DESC 
        LIMIT 10;
```
1. Database returned the records that the "command" computed.
<br><br>
1. Used the same connection to send the commands
```
show columns from People;
```
```
show columns from Batting.
```
1. The database engine returned the _metadata_ describing _Batting_ and _People._
<br><br>
1. The overall pattern is common, but the DML and DDL vary between products and engines.

__REST Model__

<img src="../images/L3_REST.jpeg">

- We will need to support the following paths:
    - /api/baseball/$<resource>$/$<id>$?fields="f1,f2, ..."
    - /api/baseball/$<resource>$/metadata
    - /api/baseball/$<resource>$?$<query>$&fields="f1, f2, ..."


- Where
    - $<resource>$ is one of players, batting, appearances
    - $<id>$ is the primary key for the resource.


## Relational and Common Database Concepts

- Almost all database engines and models have the concepts of
    - Objects that are some form of array of (name, value) pairs.
    - Sets of similar or related objects.
    - Four basic (CRUD) operations on a set
        - CREATE a new object and add to a set.
        - RETRIEVE an object in a set based on a criteria.
        - UPDATE an object in a set, e.g. change the data in the object.
        - DELETE an object from a set, specifying the object(s) by some criteria.
        
        
- In the file systems/CSV model
    - A set is a file, e.g. students.csv.
    - Each object is a row in the file.
    - The header row gives the names of each column.
    - The CRUD processing involves writing a program that reads the file, changes the two-dimensional array and writing the file.
        - CREATE: Append a row and save the file.
        - RETRIEVE: Scan the table and apply me kind of IF statement.
        - UPDATE: Change a row in the two dimensional array.
        - DELETE: Remove a row from the array.
        

- In REST/HTTP, 
    - The set is a collection of resources, e.g. /baseball/players.
    - An individual entity has an identifier relative to the collection URL, e.g. /baseball/players/willite01
    - There is not explicit metadata defining the fields and types.
    - The CRUD operations are
        - Create is POST /baseball/players { ... JSON ...}
        - Retrieve is GET, e.g. GET /baseball/players/willite01, GET /baseball/players?nameLast=Williams&nameFirst="Ted"
        - Update is PUT, e.g. PUT /baseball/players/willite01 { ... JSON ... }
        - Delete is DELETE, e.g. DELETE /baseball/players/willite01
        
        
- In the "pure" relational model
    - A set is a _relation_.
    - An object is a _row_ or _tuple_.
    - There is no support for CREATE, UPDATE or DELETE.
    - There is an _algebra_ and language for producing a new relation from existing relations that implements a form of RETRIEVE.
    
    
- In SQL,
    - A set is a _table_.
    - An object is a _row_ or _tuple_.
    - INSERT is the create operation.
    - UPDATE is the delete operation.
    - DELETE is the delete operation.
    - SELECT is the statement that realizes the relational _algebra_.
    

## Algebraic Query Language

Reference: Ramkrishan and Gherke, section 2.4.

### Algebra

"... abstract algebra (occasionally called modern algebra) is the study of algebraic structures." (https://en.wikipedia.org/wiki/Abstract_algebra) 

"In mathematics, and more specifically in abstract algebra, an algebraic structure is a set (called carrier set or underlying set) with one or more operations defined on it that satisfies a list of axioms." (https://en.wikipedia.org/wiki/Algebraic_structure)

__Group is an Example__ (http://mathworld.wolfram.com/Group.html)

"A group G is a finite or infinite set of elements together with a binary operation (called the group operation) that together satisfy the four fundamental properties of closure, associativity, the identity property, and the inverse property. The operation with respect to which a group is defined is often called the "group operation," and a set is said to be a group "under" this operation. Elements A, B, C, ... with binary operation between A and B denoted AB form a group if

1. Closure: If A and B are two elements in G, then the product AB is also in G.

2. Associativity: The defined multiplication is associative, i.e., for all A,B,C in G, (AB)C=A(BC).

3. Identity: There is an identity element I (a.k.a. 1, E, or e) such that IA=AI=A for every element A in G.

4. Inverse: There must be an inverse (a.k.a. reciprocal) of each element. Therefore, for each element A of G, the set contains an element B=A<sup>-1</sup> such that AA<sup>-1</sup>=A<sup>-1</sup>A=I.

### Why a Special Query Language? Why a Formal Language?

- Computing has a formal mathemetical model.

- Programming languages derive from the model and have their own formal definition.

- Almost every time someone publishes a new language, my reaction is, "What? Why another language? Can't we just pick one, use it and get it right?"


- Relational algebra is less powerful and expressive than Java, C, ... and other programming languages.


- The simplicity and constrained capabilities is actually the core of the value, enabling
    - Vastly simplified programming and supporting tools that yields increased productivity.
    - Development of algorithms that process the data definitions and query statements to automatically produce optimal execution plans, which are better than what a programmer can directly code.
    
    
- We will see these benefits in coming lectures.


_Simple Tool Example_ that enables "citizen programmers" to maniuplate data.

<img src="../images/ss_1.jpeg">

<br><br>
_Simple Tool Example_ for enabling business professionals to analyze and report on data.
<img src="../images/quicksite.jpeg" width="85%">

### Relational Algebra

#### Overview

- There are two notations or representations of the algebra:
    - The original, formal theory.
    - SQL
    
__Original, Formal Notation__

- The "set" in the relational algebra is the set of _relations_.


- The operations are:
    - Common set operations:
        - Union: $\cup$
        - Intersection: $\cap$
        - Difference: $-$
    - Projection: $\pi$
    - Selection: $\sigma$
    - Cartesian Product: $\times$
    - Join: $\bowtie$
    - Rename/Alias
    

- The formal notation does not support create, update or delete. You could emulate the operations by
    - Defining new relations containing the created, updated or delete tuples.
    - Using $\cup$, $\cap$, $-$ on the original relation and created/deleted/updated tuple relation.
    
    
__SQL Notation__

- The "set" in the relational algebra is the catalog of _tables_.

- The operations are:
    - Common set operations:
        - Union: UNION
        - Intersection: INTERSECT
        - Difference: EXCEPT
    - Selection, Projection, Cartesian Product, Rename/Alias and Join are clauses within a SELECT statement.
    
    ```SELECT <project clause> FROM <table> [JOIN <table> [ON <join condition]] WHERE <select condition>```
    <br><br>
    
- SQL supports additional operations, e.g
    - GROUP BY
    - ORDER BY
    


### Overview

Ramakrishnan and Gehrke, section 4.2

- The operations on an Entity Set are:
    - Common set operations:
        - Union: $\cup$
        - Intersection: $\cap$
        - Difference: $-$
    - Projection: $\pi$
    - Selection: $\sigma$
    - Cartesian Product: $\times$
    - Join: $\bowtie$
    - Rename/Alias
    
    
- Union, Intersection, Difference are the normal set operations.
   
   
   
- <span style="color: red;">Projection</span> produces a new relation that
    - Has the same rows as the original table
    - But only containing the requested columns/fields
    
    
- <span style="color: red;">Selection</span> produces a new relation that
    - Has the same columns as the original relation
    - But only contains rows with column values matching a predicate.
    
    
- <span style="color: red;">Projection, Selection and Keys are required for phase 1 of the web solution.</span>


- We will cover the other operations in the next few lectures.


### Selection

- The selection operators are: $\lt, \gt, =, \ne, \ge, \le$


- The selection operator is $\sigma$


- $\sigma(Players)$ selects all of the rows/tuples in the player relation.


- The predicate/condition is a "subscript," e.g.
    - $\sigma$<sub>playerID=napolmi01</sub>$(Players)$ selects all players with _playerID=napolmi01_
    - $\sigma$<sub>((nameLast=Williams)$\land(throws=L))\lor(birthYear\ne1914)$</sub>$(Players)$ selects all players with
        - Last name williams who threw lefthanded
        - Bith year now equal to 1914
        
        
- This notation is a little clunky. You can think of the selection predicate being similiar to what goes in an _if()_ statement if you were looping through an array testing for a match.


- __Note:__ We also needed $\sigma$ to implement GET /players/< playerid >.
    - I glossed over this fact.
    - There is nothing special about primary (or any keys) relative to selection syntanx.
    - _Keys have a profound impact on performance and data integrity, however._
    

### Projection

- The projection operator is $\pi$


- The requested columns are subscripts on the operator, e.g. $\pi$<sub>$nameLast,nameFirst,throws$</sub>$(Players)$ returns a table
    - Containing the $nameLast, nameFirst, throws$ column values, in that order
    - For all tuples in the $Players$ table.

### This is an Algebra

- The operators $\sigma$ and $\pi$ operate on relations/tables and produce relations/tables.


- This means that you can combine them into complex combinations just like any algebra.


- For example
    - T<sub>$1$</sub>$ = \pi$<sub>$nameLast,nameFirst,throws$</sub>$($$\sigma$<sub>((nameLast=Williams)$\land(throws=L))\lor(birthYear\ne1914)$</sub>$(Players)$$)$
    - T<sub>$2$</sub>$ = \sigma$<sub>((nameLast=Williams)$\land(throws=L))\lor(birthYear\ne1914)$</sub>$(\pi$<sub>$nameLast,nameFirst,throws$</sub>$(Players))$
    - Are both valid algebraic statements
    - __And__ $T$<sub>$1$</sub>$ = T$<sub>$1$</sub> because $\sigma$ and $\pi$ are commutative if $\pi$ returns all columns that $\sigma$ tests.


### Keys

- "_Data integrity_ is the maintenance of, and the assurance of the accuracy and consistency of, data over its entire life-cycle, and is a critical aspect to the design, implementation and usage of any system which stores, processes, or retrieves data.


- _Relational (integrity) constraints_ are central to the value of relational databases.
    - The database designer _declares (defines)_ integrity constraints.
    - The database management system _rejects_ any _create, update_ or _delete_ operation that would result in a constraint violation.


- _Key(s)_ is a core  _(integrity) constraint_ enforcing _data integrity._ 
    - A _super key_ is a combination of columns with the property that now two rows have the same values for the fields of a super key.
    - A _candidate key_ is a minimal _super key_, that is removing a column from the key definition means that the key no longer uniquely identifies a row.,
    - The _primary key_ is a candidate key subjectively chosen as the "best key" for uniquely identifying the tuples.
    
    
- You will some times here the term _functionally determines._
    - Assume we have a relation $R(a,b,c,d,e)$.
    - If $(a,b)$ is a key $k$, then
    - The we can say that $k(a,b)$ _functionally determines_ $(c,d,e).$
    - Given values $(x,y)$ for $(a,b)$ we can functionally return $(c,d,e).$ The function is
    
    $\pi$<sub>$c,d,e$</sub>$(\sigma$<sub>$(a=x)\land(b=y)$</sub>$(R))$


- Consider the following snapshot of the CS courses table

<img src="../images/L4_courses.jpeg">

<br>
- The underlying relation $C$ has the following fields
    - callNumber
    - courseTitle
    - courseNumber
    - courseSection
    - term
    - year
    - instructor
    - days
    - time
    
    
- Two candidate keys are
    - _(callNumber)_
    - _(courseNumber, courseSection, year, semester)_
    

### SQL SELECT

#### Overview

- MySQL [SELECT syntax](https://dev.mysql.com/doc/refman/5.7/en/select.html)
    - Other relational database management systems are very similiar)
    - And there is a core standard for the [SQL language.](https://www.iso.org/committee/45342/x/catalogue/p/1/u/0/w/0/d/0)

```
SELECT
    [ALL | DISTINCT | DISTINCTROW ]
      [HIGH_PRIORITY]
      [STRAIGHT_JOIN]
      [SQL_SMALL_RESULT] [SQL_BIG_RESULT] [SQL_BUFFER_RESULT]
      [SQL_CACHE | SQL_NO_CACHE] [SQL_CALC_FOUND_ROWS]
    select_expr [, select_expr ...]
    [FROM table_references
      [PARTITION partition_list]
    [WHERE where_condition]
    [GROUP BY {col_name | expr | position}
      [ASC | DESC], ... [WITH ROLLUP]]
    [HAVING where_condition]
    [ORDER BY {col_name | expr | position}
      [ASC | DESC], ...]
    [LIMIT {[offset,] row_count | row_count OFFSET offset}]
    [PROCEDURE procedure_name(argument_list)]
    [INTO OUTFILE 'file_name'
        [CHARACTER SET charset_name]
        export_options
      | INTO DUMPFILE 'file_name'
      | INTO var_name [, var_name]]
    [FOR UPDATE | LOCK IN SHARE MODE]]
```

- The SELECT statement is complex because it is the implementation foundation for
    - Selection
    - Projection
    - Join
    - Cross-product
    - Alias/renaming.
    

- The easiest way to get started is with examples and practice.

#### GET by Primary Key



In [1]:
import pymysql.cursors
import json


def connect():
    connection = pymysql.connect(host='localhost',
                                 user='dbuser',
                                 password='dbuser',
                                 db='lahman2016',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

def disconnect(c):
    c.close()

def pretty_print(r):
    r = json.dumps(r, indent=4, sort_keys=True)
    print("Result = ", r)

try:
    print("Get by primary key.")
    sql1 = "SELECT nameLast FROM Master WHERE playerID = 'napolmi01';"
    print("SQL statement = ", sql1)
    
    connection = connect()

    with connection.cursor() as cursor:
        # Execute the SQL statement.
        print("Connected.")
        cursor.execute(sql1)
        
        # We know that this is a primary key. We only need to fetch ONE row.
        result = cursor.fetchone()
        
        pretty_print(result)
        
except:  
    print("Something bad happened.")
finally:
    disconnect(connection)
    print("Disconnected")


Get by primary key.
SQL statement =  SELECT nameLast FROM Master WHERE playerID = 'napolmi01';
Connected.
Result =  {
    "nameLast": "Napoli"
}
Disconnected


In [2]:
try:
    print("Get by primary key.")
    sql2 = "SELECT nameLast FROM Batting WHERE yearID = " 
    sql2 = sql2 + "%s AND playerID=%s and stint=%s;"
    print("SQL statement = ", sql2)
    
    connection = connect()

    with connection.cursor() as cursor:
        # Execute the SQL statement.
        print("Connected.")
        cursor.execute(sql2,('1959','willite01', 1))
        
        # We know that this is a primary key. We only need to fetch ONE row.
        result = cursor.fetchone()
        
        pretty_print(result)
        
except:  
    print("Something bad happened.")
finally:
    disconnect(connection)
    print("Disconnected")

Get by primary key.
SQL statement =  SELECT nameLast FROM Batting WHERE yearID = %s AND playerID=%s and stint=%s;
Connected.
Something bad happened.
Disconnected


- Database connectors usually support _parameterized query statements._
    - SQL string with %s in various places.
    - (v1, ..., vn) to insert into the SQL statement sent to the database.
    - Generally is the preferred approach and also support [_prepared statements_](https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursorprepared.html), which most databases support.

#### More General SELECT

##### Find All Players with Last name Williams and Bats with One Hand and Throws with Another, and Who Was Born before 1950

In [3]:
import pymysql.cursors
import json


def connect():
    connection = pymysql.connect(host='localhost',
                                 user='dbuser',
                                 password='dbuser',
                                 db='lahman2016',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

def disconnect(c):
    c.close()

def pretty_print(r):
    r = json.dumps(r, indent=4, sort_keys=True)
    print("Result = ", r)
    
def execute_query(q):

    try:
        print("Executing query = ",q )

        connection = connect()

        with connection.cursor() as cursor:
            # Execute the SQL statement.
            print("Connected.")
            cursor.execute(q)

            # We know that this is a primary key. We only need to fetch ONE row.
            result = cursor.fetchall()
            return result;
    except:  
        print("Something bad happened.")
        return ([ { "Error ": "Error" }])
    finally:
        disconnect(connection)
        print("Disconnected")
        
        
q = "SELECT * FROM Master WHERE "
q = q + "nameLast='Williams' and NOT ( bats = throws ) "
q = q + "AND birthYear < '1950';"
r = execute_query(q)
pretty_print(r)

Executing query =  SELECT * FROM Master WHERE nameLast='Williams' and NOT ( bats = throws ) AND birthYear < '1950';
Connected.
Disconnected
Result =  [
    {
        "bats": "R",
        "bbrefID": "williac01",
        "birthCity": "Montclair",
        "birthCountry": "USA",
        "birthDay": 18,
        "birthMonth": 3,
        "birthState": "NJ",
        "birthYear": 1917,
        "deathCity": "Fort Myers",
        "deathCountry": "USA",
        "deathDay": "16",
        "deathMonth": "9",
        "deathState": "FL",
        "deathYear": "1999",
        "debut": "1940-07-15",
        "finalGame": "1946-04-22",
        "height": 74,
        "nameFirst": "Ace",
        "nameGiven": "Robert Fulton",
        "nameLast": "Williams",
        "playerID": "williac01",
        "retroID": "willa103",
        "throws": "L",
        "weight": 174
    },
    {
        "bats": "L",
        "bbrefID": "williar01",
        "birthCity": "Somerville",
        "birthCountry": "USA",
        "birthDay

#### Where Clause

__Comments__


- There [MySQL WHERE condition syntax](https://dev.mysql.com/doc/refman/5.7/en/expressions.html) (which is remarkably unhelpful) is

```
expr:
    expr OR expr
  | expr || expr
  | expr XOR expr
  | expr AND expr
  | expr && expr
  | NOT expr
  | ! expr
  | boolean_primary IS [NOT] {TRUE | FALSE | UNKNOWN}
  | boolean_primary

boolean_primary:
    boolean_primary IS [NOT] NULL
  | boolean_primary <=> predicate
  | boolean_primary comparison_operator predicate
  | boolean_primary comparison_operator {ALL | ANY} (subquery)
  | predicate

comparison_operator: = | >= | > | <= | < | <> | !=

predicate:
    bit_expr [NOT] IN (subquery)
  | bit_expr [NOT] IN (expr [, expr] ...)
  | bit_expr [NOT] BETWEEN bit_expr AND predicate
  | bit_expr SOUNDS LIKE bit_expr
  | bit_expr [NOT] LIKE simple_expr [ESCAPE simple_expr]
  | bit_expr [NOT] REGEXP bit_expr
  | bit_expr

bit_expr:
    bit_expr | bit_expr
  | bit_expr & bit_expr
  | bit_expr << bit_expr
  | bit_expr >> bit_expr
  | bit_expr + bit_expr
  | bit_expr - bit_expr
  | bit_expr * bit_expr
  | bit_expr / bit_expr
  | bit_expr DIV bit_expr
  | bit_expr MOD bit_expr
  | bit_expr % bit_expr
  | bit_expr ^ bit_expr
  | bit_expr + interval_expr
  | bit_expr - interval_expr
  | simple_expr

simple_expr:
    literal
  | identifier
  | function_call
  | simple_expr COLLATE collation_name
  | param_marker
  | variable
  | simple_expr || simple_expr
  | + simple_expr
  | - simple_expr
  | ~ simple_expr
  | ! simple_expr
  | BINARY simple_expr
  | (expr [, expr] ...)
  | ROW (expr, expr [, expr] ...)
  | (subquery)
  | EXISTS (subquery)
  | {identifier expr}
  | match_expr
  | case_expr
  | interval_expr
```


- All RDBs have similar and approximately equivalent functionality and syntax.


- "How do you get to be good at this? The same way you get to Carnegie Hall. __Practice.__"
    - There are some good online tutorials.
    - We will go through a lot of examples in class.
    - CAs and I will help if you have trouble on homework or take home exams.
    

- The SQL statement provides an overview of the condition syntax

```
SELECT * FROM Master WHERE nameLast='Williams' and NOT ( bats = throws ) AND  (deathYear - birthYear) > 80;
```

- The basic element is $term$  $operand$  $term$ or $operand(column)$
    - A term is a literal, column name or computed expression.
    - The operands are
        - $\lt$, $\leq$, $=$, $\gt$, $\ge$, $<>$ and have the obvious meanings.
        - $LIKE$ returns true if the string operand matches a [pattern.](https://dev.mysql.com/doc/refman/5.7/en/pattern-matching.html)
        - $term$ $BETWEEN$ $operand$ AND $OPERAND$ returns true of the $term$ is in the range.
    - There are additional operands and different RDBS have different extensions and modifications.
    
    

In [4]:
q = "SELECT * FROM Master WHERE nameLast LIKE 'willi%';"
r = execute_query(q)
pretty_print(r)

Executing query =  SELECT * FROM Master WHERE nameLast LIKE 'willi%';
Connected.
Disconnected
Result =  [
    {
        "bats": "R",
        "bbrefID": "williac01",
        "birthCity": "Montclair",
        "birthCountry": "USA",
        "birthDay": 18,
        "birthMonth": 3,
        "birthState": "NJ",
        "birthYear": 1917,
        "deathCity": "Fort Myers",
        "deathCountry": "USA",
        "deathDay": "16",
        "deathMonth": "9",
        "deathState": "FL",
        "deathYear": "1999",
        "debut": "1940-07-15",
        "finalGame": "1946-04-22",
        "height": 74,
        "nameFirst": "Ace",
        "nameGiven": "Robert Fulton",
        "nameLast": "Williams",
        "playerID": "williac01",
        "retroID": "willa103",
        "throws": "L",
        "weight": 174
    },
    {
        "bats": "R",
        "bbrefID": "willial02",
        "birthCity": "Valhermosa Springs",
        "birthCountry": "USA",
        "birthDay": 11,
        "birthMonth": 5,
      

### Project Clause (the Clause that Follows SELECT)

- AKA _Select Expression_


- The basic syntax is $term, term,...$ where $term$ is a column name or function of column names.


- $term $ $AS $ $alias$ renames a term (result column) in the result table.


In [5]:
q = "SELECT nameLast as last_name, nameFirst as first_name, "
q = q + "(deathYear - birthYear) AS lifespan, " 
q = q + "CONCAT(nameGiven,' ',nameLast) as greatest_hitter_ever FROM Master "
q = q + "WHERE nameLast='Williams' AND nameFirst='Ted';"
r = execute_query(q)
pretty_print(r)

Executing query =  SELECT nameLast as last_name, nameFirst as first_name, (deathYear - birthYear) AS lifespan, CONCAT(nameGiven,' ',nameLast) as greatest_hitter_ever FROM Master WHERE nameLast='Williams' AND nameFirst='Ted';
Connected.
Disconnected
Result =  [
    {
        "first_name": "Ted",
        "greatest_hitter_ever": "Theodore Samuel Williams",
        "last_name": "Williams",
        "lifespan": 84.0
    }
]


### Keys and Indexes

- Relational theory treats keys as an _integrity constraint._
    - The ICs ensure that create, update and delete do not cause data integrity.
    - This is a very important capability, and RDBs have advanced IC functionality, which we will see in future lectures.
    
    
- For read, most RDBs implement $keys$ using an _index._
    - A table can have many indexes.
    - The indexes optimize both constraint implementation, _and SELECT performance._
    - An index may or may not enforce uniqueness.
    
    
- The CREATE statement below
    - Has a _primary key_ on _playerID,_ which enforces uniqueness and optimizes SELECT.
    - A _non-unique index_ on _nameLast,_ which optimizes SELECT performance
        
```    
CREATE TABLE `Master` (
  `playerID` varchar(255) NOT NULL,
  `birthYear` int(11) DEFAULT NULL,
  `birthMonth` int(11) NOT NULL,
  `birthDay` int(11) DEFAULT NULL,
  `birthCountry` varchar(255) DEFAULT NULL,
  `birthState` varchar(255) DEFAULT NULL,
  `birthCity` varchar(255) DEFAULT NULL,
  `deathYear` varchar(255) DEFAULT NULL,
  `deathMonth` varchar(255) DEFAULT NULL,
  `deathDay` varchar(255) DEFAULT NULL,
  `deathCountry` varchar(255) DEFAULT NULL,
  `deathState` varchar(255) DEFAULT NULL,
  `deathCity` varchar(255) DEFAULT NULL,
  `nameFirst` varchar(255) NOT NULL,
  `nameLast` varchar(255) NOT NULL,
  `nameGiven` varchar(255) DEFAULT NULL,
  `weight` int(11) DEFAULT NULL,
  `height` int(11) DEFAULT NULL,
  `bats` varchar(255) DEFAULT NULL,
  `throws` varchar(255) DEFAULT NULL,
  `debut` varchar(255) DEFAULT NULL,
  `finalGame` varchar(255) DEFAULT NULL,
  `retroID` varchar(255) DEFAULT NULL,
  `bbrefID` varchar(255) DEFAULT NULL,
  PRIMARY KEY (`playerID`),
  KEY `player_idx` (`playerID`),
  KEY `name_l` (`nameLast`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
```

- The SQL Workbench is a good way to view and edit indexes.

<img src="../images/L4_index_workbench.jpeg">


Execution Examples:

<img src="../images/L4_index_performance.jpeg">

- SELECTs that cannot use an index have to scan the table, which is $O(N).$


- SELECTs that can use an index are
    - $O(N*Log(N))$ if it can use a tree index.
    - $O(1)$ if it can use a hash index.
    
    
- We will cover indexes, performance implications and  implementation in future lectures.


- The obvious question is, "Why not index everything?" Indexes take up storage, and there is a [space-time tradeoff.](https://en.wikipedia.org/wiki/Space%E2%80%93time_tradeoff)

### SQL is an Algebra

- All SQL DML statements produce a relation/table.


- This means, and it is normally the case, that SQL statements are algebraic combinations of simpler SQL statements.

```
SELECT
	playerID, nameLast, nameFirst
FROM
	(SELECT * FROM Master WHERE nameLast LIKE 'William%') as a;
```

- SELECTS all players with nameList LIKE 'William%' to produce derived table a.


- PROJECTS on a to get columns playerID, nameLast, nameFirst.


- And is equivalent to 

```
SELECT
	*
FROM
	(SELECT playerID, nameLast, nameFirst FROM Master ) as a
WHERE
	nameLast LIKE 'William%'
```

## Implementation

### Conceptual Data Model

Reminder
<br><br>
<img src="../images/conceptuallogicalphysical.jpeg">

- There are three entities
    - _Master_ represents information about an individual in the database.
    - _Appearances_ represents information about a person's appearances in games.
    - _Batting_ represents information about a player's batting for teams and seasons.
   
   
- How do you identify entity types/entities? From http://www.agiledata.org/essays/dataModeling101.html
    - "An entity type, also simply called entity (not exactly accurate terminology, but very common in practice), is similar conceptually to object-orientation’s concept of a class – an entity type represents a collection of similar objects.  An entity type could represent a collection of people, places, things, events, or concepts. Examples of entities in an order entry system would include Customer, Address, Order, Item, and Tax. If you were class modeling you would expect to discover classes with the exact same names. However, the difference between a class and an entity type is that classes have both data and behavior whereas entity types just have data. 
    - "Ideally an entity should be normal, the data modeling world’s version of cohesive. A normal entity depicts one concept, just like a cohesive class models one concept. For example, customer and order are clearly two different concepts; therefore it makes sense to model them as separate entities." 


- How do you identity relationships?
    - "In the real world entities have relationships with other entities.  For example, customers PLACE orders, customers LIVE AT addresses, and line items ARE PART OF orders. Place, live at, and are part of are all terms that define relationships between entities.  The relationships between entities are conceptually identical to the relationships (associations) between objects."  
 

<img src="../images/L3_baseball_conceptual.jpeg">

### Logical Data Model

##### Overview

- The logical data model requires adding:
    - Attributes
    - Primary Keys
    - Foreign Keys
    
##### Attributes

Identifying attributes (http://www.agiledata.org/essays/dataModeling101.html)

- "Each entity type will have one or more data attributes.  For example, 
    - ... [a] Customer entity has attributes such as First Name and Surname and ... 
    - the TCUSTOMER table had corresponding data columns CUST_FIRST_NAME and CUST_SURNAME (a column is the implementation of a data attribute within a relational database). 
    
    
- Attributes should also be cohesive from the point of view of your domain, something that is often a judgment call. ... ... 
    - we decided that we wanted to model the fact that people had both first and last names instead of just a name (e.g. “Scott" and “Ambler" vs. “Scott Ambler")
    - we did not distinguish between the sections of an American zip code (e.g. 90210-1234-5678).
    
    
- Getting the level of detail right can have a significant impact on your development and maintenance efforts.
    - Refactoring a single data column into several columns can be difficult, ...
    - over-specifying an attribute (e.g. having three attributes for zip code when you only needed one) can result in overbuilding your system and hence you incur greater development and maintenance costs than you actually needed.
    

- In our scenario,
    - We were given the data, which partially defined the attributes.
    - We could have, and will, re-factor how the given data fits into a good data model.

<br><br>
    

<img src="../images/masterlogical.jpeg">
<br>

<img src="../images/battinglogical.jpeg" width="80%">

<img src="../images/appearanceslogical.jpeg">

##### Keys and Primary Keys

Ramakrishnan and Gehrke, 2.4.1, 3.2

_Relational Theory_

(Entity) keys refers to a set of attributes that uniquely defines an entity in an entity set. Entity keys can be _super,_ _candidate_ or _primary._
- _Super key:_ A set of attributes (one or more) that together define (uniquely identify) an entity in an entity set.
- _Candidate key:_ A minimal super key, meaning it has the least possible number of attributes to still be a super key. An entity set may have more than one candidate key.
- _Primary key:_ A candidate key chosen by the database designer to uniquely identify the entity set.

In our data model,
- _Master_ primary key is _playerID_
- _Batting_ is more complicated.
    - _playerID_ does not uniquely identify a row. Players play for many years.
    - _(playerID, yearID)_ does not uniquely identify a row. A player could get traded, and play for more than a single team in a year.
    - No problem, we can use _(playerID, yearID, teamID)._ But, a player can have more than one stint with a team in a year.
    - The answer is _(playerID, yearID, stint)._
        - How do I know this? I understand baseball.
        - What if you or I do not understand the domain? We are typically working with a domain expert and these decisions are part of collaborative design in the local modeling phase.
- _Appearances_ primary key is _(playerID, yearID, teamID)._

_An aside:_ I ran the following queries for batting
```
-- (1) What is the maximum number of rows for a given playerID? Also, look up the names. 
SELECT playerID,
	(SELECT nameLast FROM Master WHERE Master.playerID=Batting.playerID) as nameLast,
    (SELECT nameFirst FROM Master WHERE Master.playerID=Batting.playerID) as nameFirst,
    count(*) as row_count FROM batting GROUP BY playerID,nameFirst,nameLast
	ORDER BY row_count DESC LIMIT 1;

-- (2) What is the maximum number of rows if I try playerID and yearID for a primary key? Also, look up the names. 
SELECT playerID,
	(SELECT nameLast FROM Master WHERE Master.playerID=Batting.playerID) as nameLast,
    (SELECT nameFirst FROM Master WHERE Master.playerID=Batting.playerID) as nameFirst,
    count(*) as row_count FROM batting GROUP BY playerID, yearID
	ORDER BY row_count DESC LIMIT 1;

-- (3) Same question using playerID, yearID, teamID. Also, look up the names. 
SELECT playerID,
	(SELECT nameLast FROM Master WHERE Master.playerID=Batting.playerID) as nameLast,
    (SELECT nameFirst FROM Master WHERE Master.playerID=Batting.playerID) as nameFirst,
    count(*) as row_count FROM batting GROUP BY playerID, yearID, teamID
	ORDER BY row_count DESC LIMIT 1;

-- (4) Same question using playerID, yearID, strint. Also, look up the names. 
SELECT playerID,
	(SELECT nameLast FROM Master WHERE Master.playerID=Batting.playerID) as nameLast,
    (SELECT nameFirst FROM Master WHERE Master.playerID=Batting.playerID) as nameFirst,
    count(*) as row_count FROM batting GROUP BY playerID,yearID,stint
	ORDER BY row_count DESC LIMIT 1;
```

These queries returned the following information.

| Query No. | Possible Key             | playerID  | last name | first name | row count |
|-----------|--------------------------|-----------|-----------|------------|-----------|
| 1         | playerID                 | mcguide01 | McGuire   | Deacon     | 31        |
| 2         | playerID, yearID         | chouife01 | Chouinard | Felix      | 5         |
| 3         | playerID, yearID, teamID | chouife01 | Chouinard | Felix      | 3         |
| 4         | playerID, yearID, stint  | zay01     | Zay       | William    | 1         |

What do the queries do?
- For a possible key combination.
- Count the maximum number of rows that have any common combination of keys.
- Returns the largest count.
- And provides information about one of the rows with the largest count.

Do not worry if you do not understand these queries, _you will!_ But, the queries verify that _(playerID,yearID,stint)_ is uniquely identifies a row/entry.

##### Foreign Keys

Ramakrishan and Gehrke, section 3.2.2

"In the context of relational databases, a foreign key is a field (or collection of fields) in one table that uniquely identifies a row of another table or the same table. In simpler words, the foreign key is defined in a second table, but it refers to the primary key or a unique key in the first table." (https://en.wikipedia.org/wiki/Foreign_key)

There are at least two perspectives on _foreign key:_
1. Foreign keys implement _Integrity Constraint,_ which we will cover later. A tuple in one table can exists only if the foreign key matches a primary key in another table.
2. Foreign keys define _Relationships._ I can use a foreign key to find tuples in two different tables that are related.

In our simple example
- Batting.playerID is a foreign key for Master.playerID.
- Appearances.playerID is a foreign key for Master.playerID.

We will see this in more detail in future lectures.


### SQL Data Model (Our Initial Physical Model)

The physical model requires that we add the following:


- The create table DDL statement
    - Table names
    - Column names
    - Column data types
    
    
- Instead of drawing a diagram, we will do directly in SQL DDL.


- Note: We should adjust the column types and sizes.

```
CREATE TABLE `Master` (
  `playerID` varchar(255) NOT NULL,
  `birthYear` int(11) DEFAULT NULL,
  `birthMonth` int(11) NOT NULL,
  `birthDay` int(11) DEFAULT NULL,
  `birthCountry` varchar(255) DEFAULT NULL,
  `birthState` varchar(255) DEFAULT NULL,
  `birthCity` varchar(255) DEFAULT NULL,
  `deathYear` varchar(255) DEFAULT NULL,
  `deathMonth` varchar(255) DEFAULT NULL,
  `deathDay` varchar(255) DEFAULT NULL,
  `deathCountry` varchar(255) DEFAULT NULL,
  `deathState` varchar(255) DEFAULT NULL,
  `deathCity` varchar(255) DEFAULT NULL,
  `nameFirst` varchar(255) NOT NULL,
  `nameLast` varchar(255) NOT NULL,
  `nameGiven` varchar(255) DEFAULT NULL,
  `weight` int(11) DEFAULT NULL,
  `height` int(11) DEFAULT NULL,
  `bats` varchar(255) DEFAULT NULL,
  `throws` varchar(255) DEFAULT NULL,
  `debut` varchar(255) DEFAULT NULL,
  `finalGame` varchar(255) DEFAULT NULL,
  `retroID` varchar(255) DEFAULT NULL,
  `bbrefID` varchar(255) DEFAULT NULL,
  PRIMARY KEY (`playerID`),
  KEY `player_idx` (`playerID`),
  KEY `name_l` (`nameLast`),
  KEY `name_f` (`nameFirst`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

CREATE TABLE `Appearances` (
  `yearID` int(11) NOT NULL,
  `teamID` varchar(255) NOT NULL,
  `lgID` varchar(255) DEFAULT NULL,
  `playerID` varchar(255) NOT NULL,
  `G_all` int(11) DEFAULT NULL,
  `GS` varchar(255) DEFAULT NULL,
  `G_batting` int(11) DEFAULT NULL,
  `G_defense` int(11) DEFAULT NULL,
  `G_p` int(11) DEFAULT NULL,
  `G_c` int(11) DEFAULT NULL,
  `G_1b` int(11) DEFAULT NULL,
  `G_2b` int(11) DEFAULT NULL,
  `G_3b` int(11) DEFAULT NULL,
  `G_ss` int(11) DEFAULT NULL,
  `G_lf` int(11) DEFAULT NULL,
  `G_cf` int(11) DEFAULT NULL,
  `G_rf` int(11) DEFAULT NULL,
  `G_of` int(11) DEFAULT NULL,
  `G_dh` varchar(255) DEFAULT NULL,
  `G_ph` varchar(255) DEFAULT NULL,
  `G_pr` varchar(255) DEFAULT NULL,
  PRIMARY KEY (`yearID`,`teamID`,`playerID`),
  UNIQUE KEY `ux` (`playerID`,`teamID`,`yearID`),
  KEY `player_idx` (`playerID`),
  KEY `year_idx` (`yearID`) USING BTREE,
  CONSTRAINT `playerID` FOREIGN KEY (`playerID`) REFERENCES `Master` (`playerID`) ON DELETE NO ACTION ON UPDATE NO ACTION
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

CREATE TABLE `Batting` (
  `playerID` varchar(255) NOT NULL,
  `yearID` int(11) NOT NULL,
  `stint` int(11) NOT NULL,
  `teamID` varchar(255) DEFAULT NULL,
  `lgID` varchar(255) DEFAULT NULL,
  `G` int(11) DEFAULT NULL,
  `AB` int(11) DEFAULT NULL,
  `R` int(11) DEFAULT NULL,
  `H` int(11) DEFAULT NULL,
  `2B` int(11) DEFAULT NULL,
  `3B` int(11) DEFAULT NULL,
  `HR` int(11) DEFAULT NULL,
  `RBI` int(11) DEFAULT NULL,
  `SB` int(11) DEFAULT NULL,
  `CS` int(11) DEFAULT NULL,
  `BB` int(11) DEFAULT NULL,
  `SO` int(11) DEFAULT NULL,
  `IBB` varchar(255) DEFAULT NULL,
  `HBP` varchar(255) DEFAULT NULL,
  `SH` varchar(255) DEFAULT NULL,
  `SF` varchar(255) DEFAULT NULL,
  `GIDP` varchar(255) DEFAULT NULL,
  PRIMARY KEY (`playerID`,`yearID`,`stint`),
  CONSTRAINT `batting_player` FOREIGN KEY (`playerID`) REFERENCES `Master` (`playerID`) ON DELETE NO ACTION ON UPDATE NO ACTION
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
 
```

- The core SQL column types are
<br><br>
<img src="../images/datatypes.jpg">


- All database management systems significantly extend the set of data/column types.


- The length options play a significant role in the database management systems optimization of storage use, which we will cover in future lectures.
    - INT(8), INT(11), TINY INT, ...
    - VARCHAR(16), VARCHAR(1024), ...
    

- If we have the DDL defined and have created the tables, we can _reverse engineer_ the physical model.


- The line endings have very precise meanings, which we will cover in future lectures.


<img src="../images/scenario1physical.jpeg">
    

### Web Resource Model -- Getting Started

We plan to start gathering displaying information about players. Our web resource model (URLs) with be
- /players?<query> to find players matching a template.
- /players/playerID to find a specific player.
    
_Example_

GET http://localhost:8000/baseball/api/players/napolmi01

Returns

```
{
    "playerID": "napolmi01",
    "birthYear": 1981,
    "birthMonth": "10",
    "birthDay": 31,
    "birthCountry": "USA",
    "birthState": "FL",
    "birthCity": "Hollywood",
    "deathYear": "",
    "deathMonth": "",
    "deathDay": "",
    "deathCountry": "",
    "deathState": "",
    "deathCity": "",
    "nameFirst": "Mike",
    "nameLast": "Napoli",
    "nameGiven": "Michael Anthony",
    "weight": 225,
    "height": 73,
    "bats": "R",
    "throws": "R",
    "debut": "2006-05-04",
    "finalGame": "2016-10-02",
    "retroID": "napom001",
    "bbrefID": "napolmi01"
}
```

<img src="../images/playergetpostman.jpeg">
